In [37]:
import difflib
import random
import requests
import ipywidgets as widgets
from ipywidgets import Button, Label, FloatText, HBox, VBox, Text, Output, Layout
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [32]:
import openmdao.api as om


class Paraboloid(om.ExplicitComponent):
    """
    Evaluates the equation f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3.
    """

    def setup(self):
        self.add_input('x', val=0.0)
        self.add_input('y', val=0.0)

        self.add_output('f_xy', val=0.0)

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3

        Minimum at: x = 6.6667; y = -7.3333
        """
        x = inputs['x']
        y = inputs['y']

        outputs['f_xy'] = (x - 3.0)**2 + x * y + (y + 4.0)**2 - 3.0


if __name__ == "__main__":

    # build the model
    prob = om.Problem()
    prob.model.add_subsystem('parab', Paraboloid(), promotes_inputs=['x', 'y'])

    # define the component whose output will be constrained
    prob.model.add_subsystem('const', om.ExecComp('g = x + y'), promotes_inputs=['x', 'y'])

    # Design variables 'x' and 'y' span components, so we need to provide a common initial
    # value for them.
    prob.model.set_input_defaults('x', 3.0)
    prob.model.set_input_defaults('y', -4.0)

    # setup the optimization
    prob.driver = om.ScipyOptimizeDriver()
    prob.driver.options['optimizer'] = 'COBYLA'

    prob.model.add_design_var('x', lower=-50, upper=50)
    prob.model.add_design_var('y', lower=-50, upper=50)
    prob.model.add_objective('parab.f_xy')

    # to add the constraint to the model
    prob.model.add_constraint('const.g', lower=0, upper=10.)

    prob.setup()

In [142]:
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()

prob.model.list_inputs()

prob.model.get_io_metadata().items()

11 Input(s) in 'model'

varname   val  
--------  -----
cycle
  d1
    z     |0.0|
  d2
    z     |0.0|
obj_cmp
  z       |0.0|
cycle
  d1
    x     [0.] 
obj_cmp
  x       [0.] 
cycle
  d1
    y2    [1.] 
obj_cmp
  y2      [1.] 
con_cmp2
  y2      [1.] 
cycle
  d2
    y1    [1.] 
obj_cmp
  y1      [1.] 
con_cmp1
  y1      [1.] 




dict_items([('cycle.d1.z', {'units': None, 'shape': (2,), 'size': 2, 'distributed': False, 'tags': set(), 'desc': '', 'shape_by_conn': False, 'copy_shape': None, 'has_src_indices': False, 'global_size': 2, 'global_shape': (2,), 'prom_name': 'z', 'discrete': True}), ('cycle.d2.z', {'units': None, 'shape': (2,), 'size': 2, 'distributed': False, 'tags': set(), 'desc': '', 'shape_by_conn': False, 'copy_shape': None, 'has_src_indices': False, 'global_size': 2, 'global_shape': (2,), 'prom_name': 'z', 'discrete': True}), ('cycle.d1.x', {'units': None, 'shape': (1,), 'size': 1, 'distributed': False, 'tags': set(), 'desc': '', 'shape_by_conn': False, 'copy_shape': None, 'has_src_indices': False, 'global_size': 1, 'global_shape': (1,), 'prom_name': 'x', 'discrete': True}), ('cycle.d1.y2', {'units': None, 'shape': (1,), 'size': 1, 'distributed': False, 'tags': set(), 'desc': '', 'shape_by_conn': False, 'copy_shape': None, 'has_src_indices': False, 'global_size': 1, 'global_shape': (1,), 'prom_nam

In [143]:
def multi_checkbox_widget(descriptions):
    """ Widget with a search field and lots of checkboxes """
    search_widget = widgets.Text()
    
#     options_dict = {description: widgets.Checkbox(description=description, value=False, indent=False,
#                                                  ) for description in descriptions}
#     options = [HBox([options_dict[description],], layout = Layout(margin='1000px 100px 100px 1000px;')) 
#                for description in descriptions]
    
    options_dict = {}
    options = []
    for description in descriptions:
            cb1 = widgets.Checkbox(description=description, value=False, indent=False)
            indent = 50 * description.count('.')
            box1 = HBox([cb1,], layout=Layout(margin=f'0 0 0 {indent}px'))
            options.append(box1)
            
            options_dict[description] = box1
            
    options = [options_dict[description] for description in descriptions]
    
    
    options_widget = widgets.VBox(options)

    


    multi_select = widgets.VBox([search_widget, options_widget])
#     options_widget = widgets.VBox(options, 
#                                   layout={'overflow': 'scroll', 'padding': '0px 0px 0px 1000px;', 'width':'auto'})
#     multi_select = widgets.VBox([search_widget, options_widget], 
#                                 layout={'overflow': 'scroll', 'padding': '0px 0px 0px 1000px;', 'width':'70%'})

    # Wire the search field to the checkboxes
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = [options_dict[description] for description in descriptions]
        else:
            # Filter by search field using difflib.
#             close_matches = difflib.get_close_matches(search_input, descriptions, cutoff=0.5)
            close_matches = [description for description in descriptions if search_input in description]
            print(close_matches)
            new_options = [options_dict[description] for description in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    return multi_select

# Example of using the widget

# meta = prob.model.get_io_metadata(('input',))
# meta = prob.model.get_io_metadata(iotypes=["output"] , includes='y')
# iotypes=["output"]
# meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata(iotypes=["input"]).items()}
# meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata(iotypes=["input"]).items()}
# meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata().items()}
varnames = list(meta.keys())
varnames = [ fullnames for (fullnames, opts) in prob.model.get_io_metadata().items()]

print(varnames)
widget = multi_checkbox_widget(varnames)
widget

['cycle.d1.z', 'cycle.d2.z', 'cycle.d1.x', 'cycle.d1.y2', 'cycle.d2.y1', 'obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'con_cmp1.y1', 'con_cmp2.y2', 'cycle.d1.y1', 'cycle.d2.y2', 'obj_cmp.obj', 'con_cmp1.con1', 'con_cmp2.con2', '_auto_ivc.v0', '_auto_ivc.v1']


['obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'con_cmp1.y1', 'con_cmp2.y2', 'obj_cmp.obj', 'con_cmp1.con1', 'con_cmp2.con2', '_auto_ivc.v0', '_auto_ivc.v1']
['obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'obj_cmp.obj']
['obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'obj_cmp.obj']
['obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'obj_cmp.obj']
['obj_cmp.x', 'obj_cmp.y1', 'obj_cmp.y2', 'obj_cmp.z', 'con_cmp1.y1', 'con_cmp2.y2', 'obj_cmp.obj', 'con_cmp1.con1', 'con_cmp2.con2', '_auto_ivc.v0', '_auto_ivc.v1']
['_auto_ivc.v0', '_auto_ivc.v1']
['_auto_ivc.v0', '_auto_ivc.v1']
['_auto_ivc.v0', '_auto_ivc.v1']


In [10]:
selected_options = [w.description for w in widget.children[1].children if w.value]
selected_options

['dog', 'sheep']

In [98]:
from ipywidgets import HBox, Layout, VBox
# description=description, value=False, indent=False
items = [HBox([Label(f'{i}{j}', layout=Layout(margin='0 0 0 100px')) for i in range(4)]) for j in range(4)]
# items = [HBox([widgets.Checkbox(description='gleep', value=False, indent=False, layout=Layout(margin='0 0 0 0px')) for i in range(4)]) for j in range(4)]
box = VBox(items, layout=Layout(margin='0 0 0 0px'))
box

In [88]:
from ipywidgets import widgets, Layout
from IPython.display import display

checkbox = widgets.Checkbox(value=True, disabled=True, description='gleep', layout=Layout(width='30px'))
# label = widgets.Label('description', layout=Layout(width='500px', margin='6px 0 0 0px'))
# box = widgets.HBox([checkbox, label])
# box = widgets.VBox([checkbox,])
# display(box)
checkbox

Checkbox(value=True, description='gleep', disabled=True, layout=Layout(width='30px'))

In [106]:
# works!!
from ipywidgets import HBox, Layout, VBox
cb1 = widgets.Checkbox(description='gleep', value=False, indent=False)
cb2 = widgets.Checkbox(description='gloop', value=False, indent=False)
box = VBox([cb1, cb2], layout=Layout(margin='0 0 0 100px'))
box

In [115]:
from ipywidgets import HBox, Layout, VBox
cb1 = widgets.Checkbox(description='gleep', value=False, indent=False)
box1 = HBox([cb1,], layout=Layout(margin='0 0 0 100px'))
cb2 = widgets.Checkbox(description='gloop', value=False, indent=False)
box2 = HBox([cb1,], layout=Layout(margin='0 0 0 0px'))
box = VBox([box1, box2], layout=Layout(margin='0 0 0 0px'))
box

In [151]:
def om_interact(prob, vars=None):
    rows = []
    meta = prob.model.get_io_metadata()
    meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata().items()}
    output = Output()

    @output.capture()
    def handle_value_change(change):
        print(change)
        val = change['new']
        target = change['owner'].description
        prob.set_val(target, val)
        print(f'set {target} to {val}')

    for varname, options in meta.items():
        print(varname)
        if varname in vars:
            desc = options['desc']
            units = options['units']
            val = prob.get_val(varname, units=units)
            input_box = FloatText(val, description=varname)
            input_box.observe(handle_value_change, names='value')
#             row = [Label(varname), input_box, Label(options['units'])]
            row = [ input_box, Label(options['units'])]
            rows.append(HBox(row))

    @output.capture()
    def run_handler(change):
        output.clear_output()
        if change.description == 'Run driver':
            print('Running driver')
            prob.run_driver()
            prob.list_problem_vars()
        elif change.description == 'Run model':
            print('Running model')
            prob.run_model()
            prob.model.list_outputs()
        else:
            raise ValueError('Unhandled button: ', change.description)

    run_driver_button = Button(description='Run driver')
    run_driver_button.on_click(run_handler)

    run_model_button = Button(description='Run model')
    run_model_button.on_click(run_handler)

    rows.append(HBox([run_model_button, run_driver_button]))

    rows.append(output)

    return VBox(rows)

print(type(selected_options))
print(selected_options)
# selected_options = [w.description for w in widget.children[1].children if w.value]
selected_options = [w.description for w in widget.children[1].children[0].children if w.value]

om_interact(prob, vars=selected_options)
# om_interact(prob, vars=['dog', 'y'])

<class 'list'>
['x', 'y', '_auto_ivc.v0']
z
x
y2
y1
obj
con1
con2
_auto_ivc.v0
_auto_ivc.v1


In [1]:
## Try ipytree   https://coderzcolumn.com/tutorials/python/tree-widget-in-jupyter-notebook-using-ipytree-ipywidgets
from ipytree import Tree, Node

tree = Tree(stripes=True)
node1 = Node('node1')
tree.add_node(node1)
tree

Tree(nodes=(Node(name='node1'),))

In [ ]:
def multi_checkbox_widget_old(descriptions):
    """ Widget with a search field and lots of checkboxes """
    search_widget = widgets.Text()
    
    align_kw = dict(
#         _css = (('.widget-label', 'min-width', '20ex'),),
        margin = '100px 0px 5px 112px'
    )
#     options_dict = {description: widgets.Checkbox(description=description, value=False, indent=False, **align_kw,
# #                                                   margin = '100px 0px 5px 12px',
# #                                                   layout = Layout(
# #                                                       border='solid 4px',
# #                                                       padding='100px 0px 0px 1000px;',
# #                                                       margin='1000px 100px 100px 1000px;',
# #                                                       width='auto',
# #                                                   )
#                                                  ) for description in descriptions}
    options_dict = {description: widgets.Checkbox(description=description, value=False, indent=False, **align_kw,
#                                                   margin = '100px 0px 5px 12px',
#                                                   layout = Layout(
#                                                       border='solid 4px',
#                                                       padding='100px 0px 0px 1000px;',
#                                                       margin='1000px 100px 100px 1000px;',
#                                                       width='auto',
#                                                   )
                                                 ) for description in descriptions}
    print(options_dict)
    options = [HBox([options_dict[description],], layout = Layout(margin='1000px 100px 100px 1000px;')) for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll', 'padding': '0px 0px 0px 1000px;', 'width':'auto'})
    multi_select = widgets.VBox([search_widget, options_widget], 
                                layout={'overflow': 'scroll', 'padding': '0px 0px 0px 1000px;', 'width':'70%'})

    # Wire the search field to the checkboxes
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = [options_dict[description] for description in descriptions]
        else:
            # Filter by search field using difflib.
#             close_matches = difflib.get_close_matches(search_input, descriptions, cutoff=0.5)
            close_matches = [description for description in descriptions if search_input in description]
            print(close_matches)
            new_options = [options_dict[description] for description in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    return multi_select

# Example of using the widget

# Get lots of words for our options
# words_url = 'https://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain'
# response = requests.get(words_url)
# response.raise_for_status()
# words = response.text
# words = set([word.lower() for word in words.splitlines()])
words = ['dog', 'cat', 'sheep']
# descriptions = random.sample(words, 100)

meta = prob.model.get_io_metadata()
meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata().items()}
varnames = list(meta.keys())
# for varname, options in meta.items():

# print(varnames)

# # multi_checkbox_widget(descriptions)
widget = multi_checkbox_widget(varnames)
widget